In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import sklearn
import scipy
from sklearn.preprocessing import OrdinalEncoder

In [48]:
df = pd.read_csv('./WA_Marketing-Campaign.csv')
df['Promotion'].unique() # Three different marketing promotions
df.head(5)
df.describe()

,MarketID,LocationID,AgeOfStore,Promotion,week,SalesInThousands
count,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000
mean,5.715328,479.656934,8.503650,2.029197,2.500000,53.466204
std,2.877001,287.973679,6.638345,0.810729,1.119055,16.755216
min,1.000000,1.000000,1.000000,1.000000,1.000000,17.340000
25%,3.000000,216.000000,4.000000,1.000000,1.750000,42.545000
50%,6.000000,504.000000,7.000000,2.000000,2.500000,50.200000
75%,8.000000,708.000000,12.000000,3.000000,3.250000,60.477500
max,10.000000,920.000000,28.000000,3.000000,4.000000,99.650000


In [42]:
df.head(5)

,MarketID,MarketSize,LocationID,AgeOfStore,Promotion,week,SalesInThousands
0,1,Medium,1,4,3,1,33.73
1,1,Medium,1,4,3,2,35.67
2,1,Medium,1,4,3,3,29.03
3,1,Medium,1,4,3,4,39.25
4,1,Medium,2,5,2,1,27.81


In [ ]:
marketsizes = df['MarketSize'].unique()
marketsizes[0] = 'Small'
marketsizes[1] = 'Medium'
marketsizes
sizeEncoder = OrdinalEncoder(categories=[marketsizes])
df['MarketSize'] = sizeEncoder.fit_transform(df[['MarketSize']])
df

promotion1 = df[df['Promotion'] == 1]
promotion2 = df[df['Promotion'] == 2]
promotion3 = df[df['Promotion'] == 3]
promotion1.describe()

In [56]:
# A/B testing promotion 1 and 3
# Using student's t-test
p1_std = np.std(promotion1)
p3_std = np.std(promotion3)

/opt/anaconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


In [ ]:
# ANOVA for multiple groups
# requires equal variances, so we use Levene's test to determine whether the population variances are equal
from scipy.stats import levene
p_val_lev = levene(promotion1['SalesInThousands'].to_numpy(), promotion2['SalesInThousands'].to_numpy(), promotion3['SalesInThousands'].to_numpy())
p_val_lev
# p-value is 0.28, null hypothesis in Levene's test is not rejected
# Therefore, we can conclude that the variance of the sales in thousands is equal for all three campaigns. 

LeveneResult(statistic=1.2696787527507418, pvalue=0.2817514565692093)

In [ ]:
# Because the variances are the same, we can use Fisher's ANOVA,
# as this test requires the assumption of equal variances.
from scipy.stats import f_oneway
